In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
training=pd.read_csv('train.csv').copy()
testing=pd.read_csv('test.csv').copy()
# data_top = training.head() 
# print(training.loc[0]) # accesing a row
print(training.loc[:,'Name']) # accesing a column

0       Weisz, Mrs. Leopold (Mathilde Francoise Pede)
1                   Williams, Mr. Howard Hugh "Harry"
2      Morley, Mr. Henry Samuel ("Mr Henry Marshall")
3         Palsson, Mrs. Nils (Alma Cornelia Berglund)
4                              Sutehall, Mr. Henry Jr
                            ...                      
663                               Ilett, Miss. Bertha
664                          Morrow, Mr. Thomas Rowan
665                                     Bing, Mr. Lee
666                       Strandberg, Miss. Ida Sofia
667                    Murphy, Miss. Katherine "Kate"
Name: Name, Length: 668, dtype: object


In [3]:
def count_nulls(dataset):
    return sum(dataset.isnull())

In [4]:
def get_name(my_list, position):
    # gets the first 3 character
    return my_list[position][:4]

In [5]:
training['Updated_Names']=training['Name'].str.split(',').apply(get_name, position=1)
print(training.loc[:,'Updated_Names']) # accesing a column
training.drop('Name', axis=1, inplace=True)

0       Mrs
1       Mr.
2       Mr.
3       Mrs
4       Mr.
       ... 
663     Mis
664     Mr.
665     Mr.
666     Mis
667     Mis
Name: Updated_Names, Length: 668, dtype: object


In [6]:
d=dict()
count=0
# print(len(set(training['Updated_Names'].loc[:])))
for i in set(training['Updated_Names']):
    # to define unique value for each distinct value in set of updated names
    d[i]=count/15
    # print(d[i])
    count+=1
training['Names']=training['Updated_Names'].map(d)
training.drop('Updated_Names', axis=1, inplace=True)

In [7]:
training['sex']=training['Sex'].map({"female":0, "male":1})
training.drop('Sex', axis=1, inplace=True)

In [8]:
training['pclass']=training['Pclass']/3
training.drop('Pclass', axis=1, inplace=True)

In [9]:
training.loc[training['Embarked'].isnull(), 'Embarked']='S'

In [10]:
training['embarked']=training.Embarked.map({"S":0, "C":0.5, "Q":1})
training.drop('Embarked', inplace=True, axis=1)

In [11]:
training['fare']=training.Fare/training.Fare.max()
training.drop('Fare', inplace=True, axis=1)

In [12]:
training['parch']=training['Parch']/training['Parch'].max()
training.drop('Parch', inplace=True, axis=1)

In [13]:
training['sibsp']=training['SibSp']/training['SibSp'].max()
training.drop('SibSp', inplace=True, axis=1)

In [14]:
cabins=[]
for i in training.loc[training['Cabin'].notnull(), 'Cabin']:
    cabins.append(i[0])

In [15]:
set_cabins=set(cabins)


In [16]:
dictionary_cabin=dict()
count=0
for i in set_cabins:
    dictionary_cabin[i]=count/8
    count+=1
dictionary_cabin['N']=1
dictionary_cabin

{'C': 0.0,
 'B': 0.125,
 'D': 0.25,
 'A': 0.375,
 'G': 0.5,
 'T': 0.625,
 'E': 0.75,
 'F': 0.875,
 'N': 1}

In [17]:
training.loc[training.Cabin.isnull(), 'Cabin']='N'


In [18]:
def cabin_mapping(get_list, position):
    return get_list[position][0]
training['cabin']=training.Cabin.str.split(' ').apply(cabin_mapping, position=0)
# print(training['cabin'].loc[10:20])

In [19]:
training.drop('Cabin', axis=1, inplace=True)

In [20]:
training['cabins']=training.cabin.map(dictionary_cabin)
training.drop('cabin', inplace=True, axis=1)

In [21]:
def ticket_mapping(get_ticket, position):
    if get_ticket[position]=='LINE':
        return -1
    return int(get_ticket[position])
training['ticket']=training['Ticket'].str.split(' ').apply(ticket_mapping, position=-1)
training.drop('Ticket', inplace=True, axis=1)

In [22]:
training['tickets']=training['ticket']/training.ticket.max()
training.drop('ticket', inplace=True, axis=1)

In [23]:
training.loc[(training.Age.isnull())&(training['Survived']==0), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==0), 'Age'].median()


In [24]:
training.loc[(training.Age.isnull())&(training['Survived']==1), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==1), 'Age'].median()


In [25]:
training['age']=training.Age/80.0
training.drop('Age', inplace=True, axis=1)

In [26]:
training

,Survived,Names,sex,pclass,embarked,fare,parch,sibsp,cabins,tickets,age
0,1,0.000000,0,0.666667,0.0,0.050749,0.000000,0.125,1.0,0.073651,0.3625
1,0,0.066667,1,1.000000,0.0,0.015713,0.000000,0.000,1.0,0.000795,0.3625
2,0,0.066667,1,0.666667,0.0,0.050749,0.000000,0.000,1.0,0.080822,0.4875
3,0,0.000000,0,1.000000,0.0,0.041136,0.666667,0.000,1.0,0.112826,0.3625
4,0,0.066667,1,1.000000,0.0,0.013761,0.000000,0.000,1.0,0.126422,0.3125
...,...,...,...,...,...,...,...,...,...,...,...
663,1,0.400000,0,0.666667,0.0,0.020495,0.000000,0.000,1.0,0.004800,0.2125
664,0,0.066667,1,1.000000,1.0,0.015127,0.000000,0.000,1.0,0.120150,0.3625
665,1,0.066667,1,1.000000,0.0,0.110272,0.000000,0.000,1.0,0.000516,0.4000
666,0,0.400000,0,1.000000,0.0,0.019202,0.000000,0.000,1.0,0.002435,0.2750


In [27]:
values=training.values
print(values)
from sklearn.model_selection import train_test_split
X=values[:, 1:]
Y=values[:, 0]
x_train, x_test, y_train, y_test=train_test_split(X, Y)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  7.36506458e-02 3.62500000e-01]
 [0.00000000e+00 6.66666667e-02 1.00000000e+00 ... 1.00000000e+00
  7.95146062e-04 3.62500000e-01]
 [0.00000000e+00 6.66666667e-02 1.00000000e+00 ... 1.00000000e+00
  8.08221152e-02 4.87500000e-01]
 ...
 [1.00000000e+00 6.66666667e-02 1.00000000e+00 ... 1.00000000e+00
  5.16232297e-04 4.00000000e-01]
 [0.00000000e+00 4.00000000e-01 0.00000000e+00 ... 1.00000000e+00
  2.43541695e-03 2.75000000e-01]
 [1.00000000e+00 4.00000000e-01 0.00000000e+00 ... 1.00000000e+00
  1.18410985e-01 3.50000000e-01]]


In [28]:
clf=LogisticRegression(n_jobs=-1, solver="saga", max_iter=20000, C=10, tol=0.000001, class_weight="balanced")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.7544910179640718

In [29]:
clf

LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=20000, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=1e-06, verbose=0,
                   warm_start=False)